<a href="https://colab.research.google.com/github/Risad-Raihan/ops_segmentation/blob/main/OPS_customer_segmentation_and_Rev_Volume_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
df_angel = pd.read_excel('/content/drive/My Drive/ops_segmentation-main/Angel.xlsx')

In [4]:
df_angel_p = pd.read_excel('/content/drive/My Drive/ops_segmentation-main/Angel+.xlsx')

In [5]:
df_angel_p.head(5)

,Outlets,Customer Code
0,Alim Knit (BD) Ltd.,AP0170028791
1,Alim Knit (BD) Ltd.,AP0170008641
2,Alim Knit (BD) Ltd.,AP0170002444
3,Alim Knit (BD) Ltd.,AP0170002445
4,Alim Knit (BD) Ltd.,AP0170519327


In [6]:
df_prev = pd.read_excel('/content/drive/My Drive/ops_segmentation-main/partner_last.xlsx')

In [7]:
df_prev.head(2)

,Customer Code
0,AP0180000215
1,AP0180000325


In [8]:
df_curr = pd.read_excel('/content/drive/My Drive/ops_segmentation-main/chalan_current.xlsx')

In [9]:
df_curr.head(2)

,Outlet,Customer Code,Date,Total Quantity,Total Sales
0,Renaissance Apparels Ltd.,AP0030000561,2024-09-01,0.865,1280
1,Renaissance Apparels Ltd.,AP0030000561,2024-09-01,1.000,699


In [10]:
df_angel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3063 entries, 0 to 3062
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Customer Code   3063 non-null   object
 1   Contact Number  3018 non-null   object
 2   Outlet          3063 non-null   object
 3   Customer Name   3063 non-null   object
 4   HR Code         3063 non-null   object
 5   Gender          2280 non-null   object
 6   Joining Date    3063 non-null   object
 7   tenure_days     3063 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 191.6+ KB


In [11]:
df_angel_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2377 entries, 0 to 2376
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Outlets        2377 non-null   object
 1   Customer Code  2377 non-null   object
dtypes: object(2)
memory usage: 37.3+ KB


In [12]:
df_curr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Outlet          657 non-null    object 
 1   Customer Code   657 non-null    object 
 2   Date            657 non-null    object 
 3   Total Quantity  657 non-null    float64
 4   Total Sales     657 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 25.8+ KB


In [13]:
df_prev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16693 entries, 0 to 16692
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer Code  16693 non-null  object
dtypes: object(1)
memory usage: 130.5+ KB


In [14]:
# Assuming df_curr is already loaded into your environment
# Calculate grouped data
grouped_df = df_curr.groupby(['Outlet', 'Customer Code', 'Date']).agg({
    'Total Quantity': 'sum',
    'Total Sales': 'sum'
}).reset_index()

# Display the grouped DataFrame
print(grouped_df)

# Calculate sums and counts for each unique Outlet and Date
summary_df = df_curr.groupby(['Date', 'Outlet']).agg({
    'Customer Code': 'nunique',  # count of unique Customer Codes
    'Total Quantity': 'sum',
    'Total Sales': 'sum'
}).rename(columns={'Customer Code': 'Customer Count'}).reset_index()

# Display the new summary DataFrame
print(summary_df)


                        Outlet Customer Code        Date  Total Quantity  \
0          Alim Knit (BD) Ltd.  AP0170000004  2024-09-01             3.0   
1          Alim Knit (BD) Ltd.  AP0170003572  2024-09-01             2.0   
2          Alim Knit (BD) Ltd.  AP0170014670  2024-09-01             4.0   
3       Concorde Garments Ltd.  AP0230000145  2024-09-01             5.0   
4               EcoFab Limited  AP0370001746  2024-09-01            89.0   
..                         ...           ...         ...             ...   
140     Southern Garments Ltd.  AP0100001409  2024-09-01             5.0   
141     Southern Garments Ltd.  AP0100003217  2024-09-01            66.0   
142     Southern Garments Ltd.  AP0100003258  2024-09-01             2.0   
143     Southern Garments Ltd.  AP0100022533  2024-09-01             1.0   
144  Ultimate Fashions Limited  AP0300000758  2024-09-01            37.0   

     Total Sales  
0            525  
1            465  
2            501  
3          

In [15]:
import pandas as pd

# Assuming df_curr, df_angel, and summary_df are already loaded into your environment

# Step 1: Filter Customer Codes in df_curr that exist in df_angel
filtered_codes = df_curr[df_curr['Customer Code'].isin(df_angel['Customer Code'])]['Customer Code'].unique()

# Step 2: Retrieve Outlet values for these filtered Customer Codes from df_angel
outlet_counts = df_angel[df_angel['Customer Code'].isin(filtered_codes)].groupby('Outlet').agg({
    'Customer Code': 'nunique'
}).rename(columns={'Customer Code': 'Angel'}).reset_index()

# Step 3: Merge the outlet counts with the summary_df on Outlet
summary_df = summary_df.merge(outlet_counts, on='Outlet', how='left')

# Step 4: Fill NaN values with 0 since not all Outlets might have a matching count
#summary_df['Angel'] = summary_df['Angel'].fillna(0)

# Display the updated summary DataFrame with the new 'Angel' column
print(summary_df)


          Date                                             Outlet  \
0   2024-09-01                                Alim Knit (BD) Ltd.   
1   2024-09-01                             Concorde Garments Ltd.   
2   2024-09-01                                     EcoFab Limited   
3   2024-09-01                         Evitex Dress Shirt Limited   
4   2024-09-01                            Green Smart Shirts Ltd.   
5   2024-09-01       Interfab Shirt Manufacturing Limited Unit-01   
6   2024-09-01  Interfab Shirt Manufacturing Limited Unit-02 (...   
7   2024-09-01                                    KDS IDR Limited   
8   2024-09-01                      Lida Textile & Dyeing Limited   
9   2024-09-01                            MG Niche Stitch Limited   
10  2024-09-01                          Majumder Garments Limited   
11  2024-09-01               Millenneium Textiles (Southern) Ltd.   
12  2024-09-01                          Picard Bangladesh Limited   
13  2024-09-01                    

In [16]:
summary_df.head(5)

,Date,Outlet,Customer Count,Total Quantity,Total Sales,Angel
0,2024-09-01,Alim Knit (BD) Ltd.,3,9.00,1491,1.0
1,2024-09-01,Concorde Garments Ltd.,1,5.00,190,1.0
2,2024-09-01,EcoFab Limited,1,89.00,1619,NaN
3,2024-09-01,Evitex Dress Shirt Limited,12,70.00,3170,NaN
4,2024-09-01,Green Smart Shirts Ltd.,10,110.36,13033,NaN


In [17]:
import pandas as pd

# Assuming df_curr, df_angel_p, and summary_df are already loaded into your environment

# Step 1: Filter Customer Codes in df_curr that exist in df_angel_p
filtered_codes_p = df_curr[df_curr['Customer Code'].isin(df_angel_p['Customer Code'])]['Customer Code'].unique()

# Step 2: Retrieve Outlet values for these filtered Customer Codes from df_angel_p
# Adjusting for the column name 'Outlets'
outlet_counts_p = df_angel_p[df_angel_p['Customer Code'].isin(filtered_codes_p)].groupby('Outlets').agg({
    'Customer Code': 'nunique'
}).rename(columns={'Customer Code': 'Angel_p'}).reset_index()

# Step 3: Merge the outlet counts with the summary_df on Outlet
# Note the change in merge parameter for 'Outlets'
summary_df = summary_df.merge(outlet_counts_p, left_on='Outlet', right_on='Outlets', how='left')

# Step 4: Fill NaN values with 0 since not all Outlets might have a matching count
#summary_df['Angel_p'] = summary_df['Angel_p'].fillna(0)

# Optional: You may want to drop the 'Outlets' column if it's no longer needed after the merge
summary_df.drop(columns=['Outlets'], inplace=True)

# Display the updated summary DataFrame with the new 'Angel_p' column
summary_df.head(5)


,Date,Outlet,Customer Count,Total Quantity,Total Sales,Angel,Angel_p
0,2024-09-01,Alim Knit (BD) Ltd.,3,9.00,1491,1.0,NaN
1,2024-09-01,Concorde Garments Ltd.,1,5.00,190,1.0,NaN
2,2024-09-01,EcoFab Limited,1,89.00,1619,NaN,NaN
3,2024-09-01,Evitex Dress Shirt Limited,12,70.00,3170,NaN,NaN
4,2024-09-01,Green Smart Shirts Ltd.,10,110.36,13033,NaN,NaN


In [18]:
summary_df['Angel_p'] = summary_df['Angel_p'].fillna(0)


In [19]:
summary_df['Angel'] = summary_df['Angel'].fillna(0)


In [20]:
import pandas as pd

# Assuming df_curr, df_prev, and summary_df are already loaded into your environment

# Step 1: Filter Customer Codes in df_curr that are not in df_prev
filtered_codes_new = df_curr[~df_curr['Customer Code'].isin(df_prev['Customer Code'])]['Customer Code'].unique()

# Step 2: Retrieve Outlet values for these filtered Customer Codes from df_curr
outlet_counts_new = df_curr[df_curr['Customer Code'].isin(filtered_codes_new)].groupby('Outlet').agg({
    'Customer Code': 'nunique'
}).rename(columns={'Customer Code': 'New'}).reset_index()

# Step 3: Merge the outlet counts with the summary_df on Outlet
summary_df = summary_df.merge(outlet_counts_new, on='Outlet', how='left')

# Step 4: Fill NaN values with 0 since not all Outlets might have a matching count
#summary_df['New'] = summary_df['New'].fillna(0)

# Display the updated summary DataFrame with the new 'New' column
summary_df.head(5)


,Date,Outlet,Customer Count,Total Quantity,Total Sales,Angel,Angel_p,New
0,2024-09-01,Alim Knit (BD) Ltd.,3,9.00,1491,1.0,0.0,2.0
1,2024-09-01,Concorde Garments Ltd.,1,5.00,190,1.0,0.0,NaN
2,2024-09-01,EcoFab Limited,1,89.00,1619,0.0,0.0,1.0
3,2024-09-01,Evitex Dress Shirt Limited,12,70.00,3170,0.0,0.0,1.0
4,2024-09-01,Green Smart Shirts Ltd.,10,110.36,13033,0.0,0.0,4.0


In [21]:
summary_df['New'] = summary_df['New'].fillna(0)

In [22]:
summary_df.head(5)

,Date,Outlet,Customer Count,Total Quantity,Total Sales,Angel,Angel_p,New
0,2024-09-01,Alim Knit (BD) Ltd.,3,9.00,1491,1.0,0.0,2.0
1,2024-09-01,Concorde Garments Ltd.,1,5.00,190,1.0,0.0,0.0
2,2024-09-01,EcoFab Limited,1,89.00,1619,0.0,0.0,1.0
3,2024-09-01,Evitex Dress Shirt Limited,12,70.00,3170,0.0,0.0,1.0
4,2024-09-01,Green Smart Shirts Ltd.,10,110.36,13033,0.0,0.0,4.0


In [23]:
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            18 non-null     object 
 1   Outlet          18 non-null     object 
 2   Customer Count  18 non-null     int64  
 3   Total Quantity  18 non-null     float64
 4   Total Sales     18 non-null     int64  
 5   Angel           18 non-null     float64
 6   Angel_p         18 non-null     float64
 7   New             18 non-null     float64
dtypes: float64(4), int64(2), object(2)
memory usage: 1.2+ KB


In [24]:
# Calculate the Non_Angel_Repeat column
summary_df['Non_Angel_Repeat'] = summary_df['Customer Count'] - (summary_df['Angel'] + summary_df['Angel_p'] + summary_df['New'])

# Display the updated summary DataFrame with the new 'Non_Angel_Repeat' column
summary_df.head(5)


,Date,Outlet,Customer Count,Total Quantity,Total Sales,Angel,Angel_p,New,Non_Angel_Repeat
0,2024-09-01,Alim Knit (BD) Ltd.,3,9.00,1491,1.0,0.0,2.0,0.0
1,2024-09-01,Concorde Garments Ltd.,1,5.00,190,1.0,0.0,0.0,0.0
2,2024-09-01,EcoFab Limited,1,89.00,1619,0.0,0.0,1.0,0.0
3,2024-09-01,Evitex Dress Shirt Limited,12,70.00,3170,0.0,0.0,1.0,11.0
4,2024-09-01,Green Smart Shirts Ltd.,10,110.36,13033,0.0,0.0,4.0,6.0


In [25]:
summary_df.info(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              18 non-null     object 
 1   Outlet            18 non-null     object 
 2   Customer Count    18 non-null     int64  
 3   Total Quantity    18 non-null     float64
 4   Total Sales       18 non-null     int64  
 5   Angel             18 non-null     float64
 6   Angel_p           18 non-null     float64
 7   New               18 non-null     float64
 8   Non_Angel_Repeat  18 non-null     float64
dtypes: float64(5), int64(2), object(2)
memory usage: 1.4+ KB


In [26]:
# Save DataFrame to Excel file in Colab's local environment
summary_df.to_excel('/content/SummaryDF.xlsx', index=False)
